This shit got really annoying really f'ing quickly.

In [103]:
import requests
import pandas as pd
from datetime import date
import os
import numpy as np

In [121]:
STARTING_FUNDS = 300 #USDC
FIRST_SOL_DATE = date(2024, 12, 17)
SMALL_PERC = 0.05
BIG_PERC = 0.10

In [3]:
# Kraken Pair Names: BTC is 'XBT' in Kraken's system
PAIRS = {
    "XBTUSDC": "Bitcoin (BTC)",
    "ETHUSDC": "Ethereum (ETH)",
    "SOLUSDC": "Solana (SOL)"
}

In [15]:
# Obtain all functions & vars from the actual main file
%run main.py

*Market Update (Daily MAs)*

*Bitcoin (BTC)*
• 1D MA: $90605.94 (+0.2%)
• 7D MA: $91650.13 (-0.1%)
• 30D MA: $88964.72 (+0.0%)
Interpretation: HOLD

*Ethereum (ETH)*
• 1D MA: $3101.00 (+0.6%)
• 7D MA: $3151.42 (-0.2%)
• 30D MA: $3023.46 (+0.0%)
Interpretation: HOLD

*Solana (SOL)*
• 1D MA: $136.40 (+0.4%)
• 7D MA: $137.37 (+0.2%)
• 30D MA: $128.86 (+0.1%)
Interpretation: HOLD



In [43]:
btc_data = perform_analysis(get_kraken_ohlc('XBTUSDC'))
eth_data = perform_analysis(get_kraken_ohlc('ETHUSDC'))
sol_data = perform_analysis(get_kraken_ohlc('SOLUSDC'))

In [114]:
selectdate = btc_data.iloc[0]['time']
type(selectdate)
# test = sol_data.loc[sol_data['time'] == selectdate]
# test.shape[0] == 0

pandas._libs.tslibs.timestamps.Timestamp

In [115]:
def backtest_analysis(yesterday: pd.Series, today: pd.Series):
    if np.isnan(yesterday.SMA_7) or np.isnan(today.SMA_7) \
        or np.isnan(yesterday.SMA_30) or np.isnan(today.SMA_30):
        print(f'HOLD: NaN values.')
        return Actions.HOLD
    else:
        result = determine_action(today, yesterday)
        return result['action']
    
def get_action(dataframe: pd.DataFrame, selectdate: date):
    # .iloc[0] may not be a pretty solution, but it does force the data into a series.
    yesterday = dataframe.loc[dataframe['time'] == selectdate - pd.Timedelta(days=1)].iloc[0]
    today = dataframe.loc[dataframe['time'] == selectdate].iloc[0]

    if yesterday.shape[0] == 0 or today.shape[0] == 0:
        print('HOLD: no data.')
        return Actions.HOLD
    return backtest_analysis(yesterday, today)

In [124]:
def check_values(assets_in_USDC, wallet_balance):
    if wallet_balance < 0.0:
        raise ValueError(f'Somehow achieved negative wallet balance: {wallet_balance}')
    if assets_in_USDC['BTC'] < 0.0 or assets_in_USDC['ETH'] < 0.0 or assets_in_USDC['SOL'] < 0.0:
        raise ValueError(f'Somehow achieved negative asset value: \n{assets_in_USDC}')
    

In [125]:
def buy_asset(assets_in_USDC, wallet_balance, asset_name, purchase_amount):
    wallet_balance -= purchase_amount
    assets_in_USDC[asset_name] += purchase_amount
    check_values(assets_in_USDC, wallet_balance)
    return {'assets_in_USDC': assets_in_USDC, 'wallet_balance': wallet_balance}

def sell_asset(assets_in_USDC, wallet_balance, asset_name, purchase_amount):
    assets_in_USDC[asset_name] -= purchase_amount
    wallet_balance += purchase_amount
    check_values(assets_in_USDC, wallet_balance)
    return {'assets_in_USDC': assets_in_USDC, 'wallet_balance': wallet_balance}

In [131]:
def find_value(total_value, wallet_balance, asset_value, action_type):
    # Don't try to buy/sell more than we can
    if action_type in (Actions.BUYSMALL, Actions.SELLSMALL):
        max_value = total_value * SMALL_PERC
    elif action_type in (Actions.BUYBIG, Actions.SELLBIG):
        max_value = total_value * BIG_PERC
    else:
        return 0 # Actions.HOLD

    if action_type in (Actions.BUYSMALL, Actions.BUYBIG):
        return min(max_value, wallet_balance)
    else: # action_type in (Actions.SELLSMALL, Actions.SELLBIG)
        return min(asset_value, max_value)
    # HOLD is already covered


def resolve_actions(wallet_balance, assets_in_USDC, actions):
    total_value = wallet_balance + assets_in_USDC['BTC'] + assets_in_USDC['ETH'] + assets_in_USDC['SOL']
    result = {}

    # One asset at a time, to avoid overspending/overselling.
    assets = ['BTC', 'ETH', 'SOL']
    
    for asset in assets:
        tx_value = find_value(total_value, wallet_balance, assets_in_USDC[asset], actions[asset])
        if actions[asset] in (Actions.SELLSMALL, Actions.SELLBIG):
            print(f'Selling {tx_value} USDC worth of {asset}')
            result = sell_asset(assets_in_USDC, wallet_balance, asset, tx_value)
            assets_in_USDC = result['assets_in_USDC']
            wallet_balance = result['wallet_balance']
        elif actions[asset] in (Actions.BUYSMALL, Actions.BUYBIG):
            print(f'Buying {tx_value} USDC worth of {asset}')
            result = buy_asset(assets_in_USDC, wallet_balance, asset, tx_value)
            assets_in_USDC = result['assets_in_USDC']
            wallet_balance = result['wallet_balance']

    return {'assets_in_USDC': assets_in_USDC, 'wallet_balance': wallet_balance}

In [135]:
def update_assets(assets_in_USDC):
    print('This might be important. Lol.')
    return assets_in_USDC

In [136]:
wallet_balance = STARTING_FUNDS
assets_in_USDC = {'BTC': 0, 'ETH': 0, 'SOL': 0}

for i in range(0, btc_data.shape[0]-1):
    selectdate = btc_data.iloc[i+1]['time']
    print(f'The date is {selectdate.date()}')
    assets_in_USDC = update_assets(assets_in_USDC)
    print(f'The portfolio atm is:\n\tCash:{wallet_balance}\n\tBTC:{assets_in_USDC['BTC']}\n\tETH:{assets_in_USDC['ETH']}\n\tSOL:{assets_in_USDC['SOL']}')
    print('Today we are doing:')

    btc_action = get_action(btc_data, selectdate)
    eth_action = get_action(eth_data, selectdate)
    sol_action = Actions.HOLD # Might be a no-op at the start due to later starting data

    if (selectdate.date() >= FIRST_SOL_DATE):
        sol_action = get_action(sol_data, selectdate)
        print(f'\tBTC: {btc_action.name} \n\tETH: {eth_action.name} \n\tSOL: {sol_action.name}\n')
    else:
        print(f'\tBTC: {btc_action.name}   \n\tETH: {eth_action.name} \n\tSOL: no data yet\n')

    actions = {'BTC': btc_action, 'ETH': eth_action, 'SOL': sol_action}
    result = resolve_actions(wallet_balance, assets_in_USDC, actions)
    assets_in_USDC = result['assets_in_USDC']
    wallet_balance = result['wallet_balance']

The date is 2024-01-23
This might be important. Lol.
The portfolio atm is:
	Cash:300
	BTC:0
	ETH:0
	SOL:0
Today we are doing:
HOLD: NaN values.
HOLD: NaN values.
	BTC: HOLD   
	ETH: HOLD 
	SOL: no data yet

The date is 2024-01-24
This might be important. Lol.
The portfolio atm is:
	Cash:300
	BTC:0
	ETH:0
	SOL:0
Today we are doing:
HOLD: NaN values.
HOLD: NaN values.
	BTC: HOLD   
	ETH: HOLD 
	SOL: no data yet

The date is 2024-01-25
This might be important. Lol.
The portfolio atm is:
	Cash:300
	BTC:0
	ETH:0
	SOL:0
Today we are doing:
HOLD: NaN values.
HOLD: NaN values.
	BTC: HOLD   
	ETH: HOLD 
	SOL: no data yet

The date is 2024-01-26
This might be important. Lol.
The portfolio atm is:
	Cash:300
	BTC:0
	ETH:0
	SOL:0
Today we are doing:
HOLD: NaN values.
HOLD: NaN values.
	BTC: HOLD   
	ETH: HOLD 
	SOL: no data yet

The date is 2024-01-27
This might be important. Lol.
The portfolio atm is:
	Cash:300
	BTC:0
	ETH:0
	SOL:0
Today we are doing:
HOLD: NaN values.
HOLD: NaN values.
	BTC: HOLD